In [4]:
import json

# Load Files

In [59]:
tflite_json = open('tflite_fp32_accuracy_outputs.json')
tflite = json.load(tflite_json)
# 2-d array with first element being input file and second element
# being list of dictionaries of 'box', 'class', 'score'
armnn_json = open('armnn_tflite_fp32_accuracy_outputs.json')
# Dictionary with one key ('predictions')
# Key contains list of images
# First element is input file, second element is list are dictionaries
# Dictionaries contain 'box', 'class', 'score'
# print(len(armnn['predictions'][0][1]))
armnn = json.load(armnn_json) 

# Preprocess

In [60]:
## Sort by file name
tflite = sorted(tflite, key=lambda x: (x[0]))
armnn = armnn['predictions']
armnn = sorted(armnn, key=lambda x: (x[0]))

In [82]:
print((tflite[113][1]))

[]


In [89]:
print((armnn[0][1][1]['score']))

0.710054338


# Comparison

In [105]:
# Comparison of averages scores of each image
# Also checks if there are the same number of boxes and classes
# We assume armnn is pretty comparible depending on how close the 
# average scores are, and how big the mismatch list is

error_score_list = []
different_num_boxes_list = []
num_boxes_difference_list = []

if (len(armnn) == len(tflite)):
    for i in range(len(armnn)):
        cur_tflite_input_name = tflite[i][0]
        cur_armnnn_input_name = armnn[i][0]
        
        if (cur_tflite_input_name == cur_armnnn_input_name):
            
            if (len(tflite[i][1]) != len(armnn[i][1])):
                print('tflite num boxes {}, armnn num boxes {}'.format(len(tflite[i][1]), len(armnn[i][1])))
                different_num_boxes_list.append(cur_tflite_input_name)
                num_boxes_difference_list.append(abs(len(tflite[i][1]) - len(armnn[i][1])))
                
            # Get average scores and calculate error
            if len(tflite[i][1]) != 0 and len(armnn[i][1]) != 0:
                
                # Calculate averages
                tflite_avg = 0
                for j in range(len(tflite[i][1])):
                    tflite_avg += float(tflite[i][1][j]['score'])
                tflite_avg = tflite_avg/len(tflite[i][1])
                
                armnn_avg = 0
                for j in range(len(armnn[i][1])):
                    armnn_avg += float(armnn[i][1][j]['score'])
                armnn_avg = armnn_avg/len(armnn[i][1])
                
                ## Calcualte errors
                cur_error = abs(tflite_avg - armnn_avg)/tflite_avg
                error_score_list.append(cur_error)
            
            else:
                print('tflite num boxes {}, armnn num boxes {}'.format(len(tflite[i][1]), len(armnn[i][1])))
        else:
            print('Mismatch')

tflite num boxes 3, armnn num boxes 2
tflite num boxes 6, armnn num boxes 5
tflite num boxes 7, armnn num boxes 6
tflite num boxes 4, armnn num boxes 3
tflite num boxes 8, armnn num boxes 7
tflite num boxes 8, armnn num boxes 5
tflite num boxes 5, armnn num boxes 6
tflite num boxes 9, armnn num boxes 8
tflite num boxes 4, armnn num boxes 3
tflite num boxes 5, armnn num boxes 6
tflite num boxes 5, armnn num boxes 6
tflite num boxes 0, armnn num boxes 0
tflite num boxes 0, armnn num boxes 0
tflite num boxes 4, armnn num boxes 3
tflite num boxes 5, armnn num boxes 4
tflite num boxes 6, armnn num boxes 7
tflite num boxes 0, armnn num boxes 0
tflite num boxes 2, armnn num boxes 1
tflite num boxes 7, armnn num boxes 8
tflite num boxes 0, armnn num boxes 0
tflite num boxes 4, armnn num boxes 3
tflite num boxes 0, armnn num boxes 0
tflite num boxes 5, armnn num boxes 4
tflite num boxes 3, armnn num boxes 2
tflite num boxes 3, armnn num boxes 2
tflite num boxes 6, armnn num boxes 7
tflite num b

In [98]:
print(sum(error_score_list)/len(error_score_list))

0.01632282343346439


In [101]:
print(len(different_num_boxes_list))

507


In [106]:
print(max(num_boxes_difference_list))

4


In [107]:
print(sum(num_boxes_difference_list)/len(num_boxes_difference_list))

1.1301775147928994
